<div style="background-color: #CD5C5C; padding: 20px; border-radius: 10px; box-shadow: 5px 5px 10px #888888;">
  <h1 style="color: white; font-size: 30px; font-weight: bold; text-align: center; text-shadow: 2px 2px 4px #000000;">Sports Data Analysis: Problem Statement 6
</h1>
</div>

### 1. Creation of tables using MySQL workbench  

The tables that are required for the database was created using MySQL Workbench and succesfully loaded the data from the csv file to the tables created

In [8]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="HithaChoudhary.21",
    database="sports_data",
    port='3307')
 
mycursor = mydb.cursor()
mycursor.execute("Show tables;")
myresult = mycursor.fetchall()
print("The tables present in the database:")
for x in myresult:
    print(x)

The tables present in the database:
('football_data',)
('playerdetails',)
('playermatchstats',)
('playerperformancemetrics',)


### 2. Advanced sql features like CTEs-Common Table Expression  

Executing an example usage of CTE for representing top players in the dataset

In [5]:
def create_top_players():
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="HithaChoudhary.21",
            database="sports_data",
            port='3307'
        )

        #Create cursor
        cursor = conn.cursor()
        #Define the CTE query to select top players
        cte_query = """
            WITH TopPlayer AS (
                SELECT
                    ID,
                    Player,
                    Team,
                    Goals,
                    Season,
                    ROW_NUMBER() OVER (ORDER BY Goals DESC) AS PlayerRank
                FROM
                    football_data
            )
            SELECT
               ID,
                Player,
                Team,
                Goals,
                PlayerRank
            FROM
                TopPlayer
            WHERE
                PlayerRank <= 10
        """
        cursor.execute(cte_query)
        top_players = cursor.fetchall()
        print("Top Players:")
        for player in top_players:
            print(player)

    except mysql.connector.Error as error:
        print("Error:", error)

    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
            print("MySQL connection is closed")

# Call the function to create the CTE for top players
create_top_players()
print('top_players created')

Top Players:
(15787, 'Player C', 'Team B', 50, 1)
(16047, 'Player C', 'Team C', 50, 2)
(16425, 'Player C', 'Team A', 50, 3)
(10869, 'Player B', 'Team C', 50, 4)
(10897, 'Player D', 'Team C', 50, 5)
(10944, 'Player B', 'Team A', 50, 6)
(11163, 'Player A', 'Team B', 50, 7)
(11487, 'Player B', 'Team A', 50, 8)
(12046, 'Player C', 'Team A', 50, 9)
(12562, 'Player D', 'Team A', 50, 10)
MySQL connection is closed
top_players created


### 3. Complex Analytical Queries  

Ensuring that the data stored supports complex analytical queries by executing them 

#### 3.1 Average goals per Match  
Calculating the average goals per match

In [15]:
conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="HithaChoudhary.21",
            database="sports_data",
            port='3307')
cursor = conn.cursor()
def average_goals_per_match(cursor):
    query = """
        SELECT Position, AVG(Goals) AS AvgGoalsPerMatch
        FROM football_data
        GROUP BY Position
    """
    cursor.execute(query)
    results = cursor.fetchall()
    return results

avg_goals_by_position = average_goals_per_match(cursor)
for position, avg_goals in avg_goals_by_position:
    print(f"{position}: {avg_goals:.2f} goals per match on average")
    
cursor.close()
conn.close()

Defender: 12.95 goals per match on average
Forward: 12.45 goals per match on average
Goalkeeper: 12.77 goals per match on average
Midfielder: 12.79 goals per match on average


#### 3.2 Palyer Fatigue Data  

Analyzing the player fatigue to identify player fatigue over time

In [21]:
conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="HithaChoudhary.21",
            database="sports_data",
            port='3307')
cursor = conn.cursor()
def player_fatigue(cursor):
    query = """
        SELECT AVG(PlayerFatigue) AS AvgFatigue,
            MAX(PlayerFatigue) AS MaxFatigue,
            MIN(PlayerFatigue) AS MinFatigue,
            COUNT(*) AS NumOfPlayers
        FROM 
            PlayerPerformanceMetrics 
        GROUP BY
            Player;
    """
    cursor.execute(query)
    results = cursor.fetchall()
    return results

player_fatigue_data = player_fatigue(cursor)
for row in player_fatigue_data: 
    avg_fatigue, max_fatigue, min_fatigue, num_players = row
    print(f"Avg Fatigue: {avg_fatigue}, Max Fatigue: {max_fatigue}, Min Fatigue: {min_fatigue}, Num Of Players: {num_players}")
cursor.close()
conn.close()

Avg Fatigue: 0.49435261, Max Fatigue: 0.9995, Min Fatigue: 0.0001, Num Of Players: 3800
Avg Fatigue: 0.49893683, Max Fatigue: 0.9997, Min Fatigue: 0.0004, Num Of Players: 3918
Avg Fatigue: 0.48533245, Max Fatigue: 0.9996, Min Fatigue: 0.0000, Num Of Players: 4216
Avg Fatigue: 0.49273488, Max Fatigue: 0.9994, Min Fatigue: 0.0000, Num Of Players: 3908


### 4. Data Security and Access Control Mechanisms  

In [26]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="HithaChoudhary.21",
    database="sports_data",
    port='3307'
)
cursor = conn.cursor()

#Create a new user
def create_user(username, password):
    query = f"CREATE USER '{username}'@'localhost' IDENTIFIED BY '{password}'"
    cursor.execute(query)
    print(f"User {username} created successfully")

#Grant privileges to the user
def grant_privileges(username, database):
    query = f"GRANT ALL PRIVILEGES ON {database}.* TO '{username}'@'localhost'"
    cursor.execute(query)
    print(f"Privileges granted to {username}")

#Grant specific privileges to the user
def grant_specific_privileges(username, database, privileges):
    query = f"GRANT {privileges} ON {database}.* TO '{username}'@'localhost'"
    cursor.execute(query)
    print(f"{privileges} privileges granted to {username}")
    
#Revoke privileges from the user
def revoke_privileges(username, database):
    query = f"REVOKE ALL PRIVILEGES ON {database}.* FROM '{username}'@'localhost'"
    cursor.execute(query)
    print(f"Privileges revoked from {username}")

#Revoke specific privileges from the user
def revoke_specific_privileges(username, database, privileges):
    query = f"REVOKE {privileges} ON {database}.* FROM '{username}'@'localhost'"
    cursor.execute(query)
    print(f"{privileges} privileges revoked from {username}")
    
#Drop user
def drop_user(username):
    query = f"DROP USER '{username}'@'localhost'"
    cursor.execute(query)
    print(f"User {username} dropped successfully")

#Usage example
create_user('new_user_1', '1234sql')
#Granting only the specified privileges
#Different privileges include SELECT, INSERT, UPDATE, DELETE, CREATE, DROP, ALTER, GRANT OPTION, ALL PRIVILEGES:
grant_specific_privileges('new_user_1', 'sports_data','UPDATE')

conn.commit()
conn.close()

User new_user_1 created successfully
UPDATE privileges granted to new_user_1
